
- data\raw\google_maps\metadata-sitios\1.parquet
- data\raw\google_maps\reviews-estados\review-Florida\1.parquet

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sys

# Configuration - Update these variables with your settings
MYSQL_USER = 'root'
MYSQL_PASSWORD = 'root'
MYSQL_HOST = 'localhost'
MYSQL_PORT = '3306'
MYSQL_DB = 'google_yelp'

# Paths to your Parquet files
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'

def create_db_engine():
    """Create a SQLAlchemy engine for MySQL."""
    connection_string = f'mysql+mysqlconnector://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}'
    try:
        engine = create_engine(connection_string)
        return engine
    except Exception as e:
        print(f"Error creating engine: {e}")
        sys.exit(1)

def load_parquet_to_mysql(parquet_file, table_name, engine, chunksize=1000):
    """Load a Parquet file into a MySQL table."""
    try:
        df = pd.read_parquet(parquet_file)
        # Optional: Data cleaning or transformation can be done here

        # Insert data into MySQL
        df.to_sql(name=table_name, con=engine, if_exists='append', index=False, chunksize=chunksize)
        print(f"Successfully loaded {parquet_file} into {table_name}")
    except Exception as e:
        print(f"Error loading {parquet_file} into {table_name}: {e}")


def main():
    engine = create_db_engine()

    # Load MAPS_SITES first to satisfy foreign key constraints
    load_parquet_to_mysql(MAPS_SITES_PARQUET, 'MAPS_SITES', engine)

    # Then load MAPS_REVIEWS
    load_parquet_to_mysql(MAPS_REVIEWS_PARQUET, 'MAPS_REVIEWS', engine)

if __name__ == "__main__":
    main()

: 

In [1]:
import os

print("MAPS_SITES.parquet size:", os.path.getsize('../google_maps/metadata-sitios/1.parquet') / (1024 ** 2), "MB")
print("MAPS_REVIEWS.parquet size:", os.path.getsize('../google_maps/reviews-estados/review-Florida/1.parquet') / (1024 ** 2), "MB")


MAPS_SITES.parquet size: 48.346184730529785 MB
MAPS_REVIEWS.parquet size: 12.899727821350098 MB


In [2]:
import pandas as pd

try:
    df_sites = pd.read_parquet('../google_maps/metadata-sitios/1.parquet')
    print("MAPS_SITES.parquet loaded successfully. Shape:", df_sites.shape)
except Exception as e:
    print("Error reading MAPS_SITES.parquet:", e)

try:
    df_reviews = pd.read_parquet('../google_maps/reviews-estados/review-Florida/1.parquet')
    print("MAPS_REVIEWS.parquet loaded successfully. Shape:", df_reviews.shape)
except Exception as e:
    print("Error reading MAPS_REVIEWS.parquet:", e)


MAPS_SITES.parquet loaded successfully. Shape: (275001, 15)
MAPS_REVIEWS.parquet loaded successfully. Shape: (150000, 8)


In [3]:
import mysql.connector

try:
    conn = mysql.connector.connect(
        user='root',
        password='root',
        host='localhost',
        port='3306',
        database='google_yelp'
    )
    if conn.is_connected():
        print("Connected to MySQL database")
    conn.close()
except mysql.connector.Error as err:
    print(f"Error: {err}")


: 

In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd

# Configuration - Update these variables with your settings
MYSQL_USER = 'root'
MYSQL_PASSWORD = 'root'
MYSQL_HOST = 'localhost'
MYSQL_PORT = '3306'
MYSQL_DB = 'google_yelp'

connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}'
engine = create_engine(connection_string)

# Paths to your Parquet files
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'

def create_db_engine():
    """Create a SQLAlchemy engine for MySQL."""
    connection_string = f'mysql+mysqlconnector://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}'
    try:
        engine = create_engine(connection_string)
        return engine
    except Exception as e:
        print(f"Error creating engine: {e}")
        sys.exit(1)

def load_parquet_to_mysql(parquet_file, table_name, engine, chunksize=1000):
    """Load a Parquet file into a MySQL table."""
    try:
        df = pd.read_parquet(parquet_file)
        # Optional: Data cleaning or transformation can be done here

        # Insert data into MySQL
        df.to_sql(name=table_name, con=engine, if_exists='append', index=False, chunksize=chunksize)
        print(f"Successfully loaded {parquet_file} into {table_name}")
    except Exception as e:
        print(f"Error loading {parquet_file} into {table_name}: {e}")


def main():
    engine = create_db_engine()

    # Load MAPS_SITES first to satisfy foreign key constraints
    load_parquet_to_mysql(MAPS_SITES_PARQUET, 'MAPS_SITES', engine)

    # Then load MAPS_REVIEWS
    load_parquet_to_mysql(MAPS_REVIEWS_PARQUET, 'MAPS_REVIEWS', engine)

if __name__ == "__main__":
    main()


Error loading ../google_maps/metadata-sitios/1.parquet into MAPS_SITES: (mysql.connector.errors.InterfaceError) Failed executing the operation; Python type numpy.ndarray cannot be converted
[SQL: INSERT INTO `MAPS_SITES` (name, address, gmap_id, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, `MISC`, state, relative_results, url) VALUES (%(name)s, %(address)s, %(gmap_id)s, %(description)s, %(latitude)s, %(longitude)s, %(category)s, %(avg_rating)s, %(num_of_reviews)s, %(price)s, %(hours)s, %(MISC)s, %(state)s, %(relative_results)s, %(url)s)]
[parameters: [{'name': 'Porter Pharmacy', 'address': 'Porter Pharmacy, 129 N Second St, Cochran, GA 31014', 'gmap_id': '0x88f16e41928ff687:0x883dad4fd048e8f8', 'description': None, 'latitude': 32.3883, 'longitude': -83.3571, 'category': array(['Pharmacy'], dtype=object), 'avg_rating': 4.9, 'num_of_reviews': 16, 'price': None, 'hours': array([array(['Friday', '8AM‚Äì6PM'], dtype=object),
       array(['Saturday',

In [ ]:
import pandas as pd
import numpy as np
import json


# Paths to your Parquet files
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'

def create_db_engine():
    """Create a SQLAlchemy engine for MySQL using PyMySQL."""
    connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}'
    try:
        engine = create_engine(connection_string, pool_pre_ping=True)
        return engine
    except Exception as e:
        print(f"Error creating engine: {e}")
        sys.exit(1)

def load_parquet_to_mysql(parquet_file, table_name, engine, chunksize=1000):
    """Load a Parquet file into a MySQL table."""
    try:
        print(f"Loading Parquet file: {parquet_file}")
        df = pd.read_parquet(parquet_file)
        print(f"Parquet file {parquet_file} loaded successfully. Number of rows: {len(df)}")
        
        # Identify columns with complex data types
        complex_columns = ['category', 'hours', 'MISC', 'relative_results']
        
        for col in complex_columns:
            if col in df.columns:
                df[col] = df[col].apply(convert_to_json)
        
        # Insert data into MySQL
        df.to_sql(
            name=table_name,
            con=engine,
            if_exists='append',
            index=False,
            chunksize=chunksize,
            method='multi'  # Use multi for batch inserts
        )
        print(f"Successfully loaded {parquet_file} into {table_name}\n")
    except Exception as e:
        print(f"Error loading {parquet_file} into {table_name}: {e}\n")


def convert_to_json(value):
    """Convert complex types to JSON strings."""
    if isinstance(value, np.ndarray):
        # Convert numpy array to list, then to JSON
        return json.dumps(value.tolist())
    elif isinstance(value, dict):
        # Convert dictionary with possible numpy arrays to JSON
        return json.dumps({k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in value.items()})
    elif pd.isnull(value):
        return None  # Handle NaN values
    else:
        return value  # Return the value as is if it's already a scalar

def main():
    engine = create_db_engine()

    # Load MAPS_SITES first to satisfy foreign key constraints
    load_parquet_to_mysql(MAPS_SITES_PARQUET, 'MAPS_SITES', engine)

    # Then load MAPS_REVIEWS
    load_parquet_to_mysql(MAPS_REVIEWS_PARQUET, 'MAPS_REVIEWS', engine)

if __name__ == "__main__":
    main()



In [4]:
import pandas as pd
import numpy as np
import json
from sqlalchemy import create_engine
import sys
import pymysql  # Ensure PyMySQL is installed

# Configuration - Update these variables with your settings
MYSQL_USER = 'root'
MYSQL_PASSWORD = 'root'
MYSQL_HOST = 'localhost'
MYSQL_PORT = '3306'
MYSQL_DB = 'google_yelp'

# Paths to your Parquet files
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'

def convert_to_json(value):
    """Convert complex types to JSON strings."""
    if isinstance(value, np.ndarray):
        # Convert numpy array to list, then to JSON
        return json.dumps(value.tolist())
    elif isinstance(value, dict):
        # Convert dictionary with possible numpy arrays to JSON
        return json.dumps({k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in value.items()})
    elif pd.isnull(value):
        return None  # Handle NaN values
    else:
        return value  # Return the value as is if it's already a scalar

def create_db_engine():
    """Create a SQLAlchemy engine for MySQL using PyMySQL."""
    connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}'
    try:
        engine = create_engine(connection_string, pool_pre_ping=True)
        print("Database engine created successfully.")
        return engine
    except Exception as e:
        print(f"Error creating engine: {e}")
        sys.exit(1)

def load_parquet_to_mysql(parquet_file, table_name, engine, chunksize=1000):
    """Load a Parquet file into a MySQL table."""
    try:
        print(f"Loading Parquet file: {parquet_file}")
        df = pd.read_parquet(parquet_file)
        print(f"Parquet file {parquet_file} loaded successfully. Number of rows: {len(df)}")
        
        # Convert complex data types to JSON strings
        complex_columns = ['category', 'hours', 'MISC', 'relative_results']
        for col in complex_columns:
            if col in df.columns:
                print(f"Converting column '{col}' to JSON.")
                df[col] = df[col].apply(convert_to_json)
        
        # Insert data into MySQL
        df.to_sql(
            name=table_name,
            con=engine,
            if_exists='append',
            index=False,
            chunksize=chunksize,
            method='multi'  # Use multi for batch inserts
        )
        print(f"Successfully loaded {parquet_file} into {table_name}\n")
    except Exception as e:
        print(f"Error loading {parquet_file} into {table_name}: {e}\n")

def main():
    engine = create_db_engine()

    # Load MAPS_SITES first to satisfy foreign key constraints
    load_parquet_to_mysql(MAPS_SITES_PARQUET, 'MAPS_SITES', engine)

    # Then load MAPS_REVIEWS
    load_parquet_to_mysql(MAPS_REVIEWS_PARQUET, 'MAPS_REVIEWS', engine)

if __name__ == "__main__":
    main()


Database engine created successfully.
Loading Parquet file: ../google_maps/metadata-sitios/1.parquet
Parquet file ../google_maps/metadata-sitios/1.parquet loaded successfully. Number of rows: 275001
Converting column 'category' to JSON.
Converting column 'hours' to JSON.
Error loading ../google_maps/metadata-sitios/1.parquet into MAPS_SITES: Object of type ndarray is not JSON serializable

Loading Parquet file: ../google_maps/reviews-estados/review-Florida/1.parquet
Parquet file ../google_maps/reviews-estados/review-Florida/1.parquet loaded successfully. Number of rows: 150000
Error loading ../google_maps/reviews-estados/review-Florida/1.parquet into MAPS_REVIEWS: dict can not be used as parameter



In [5]:
import pandas as pd
import numpy as np
import json
from sqlalchemy import create_engine
import sys
import pymysql  # Ensure PyMySQL is installed

# Configuration - Update these variables with your settings
MYSQL_USER = 'root'
MYSQL_PASSWORD = 'root'
MYSQL_HOST = 'localhost'
MYSQL_PORT = '3306'
MYSQL_DB = 'google_yelp'

# Paths to your Parquet files
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'

def convert_to_json(value):
    """Recursively convert complex types to JSON strings."""
    def recursive_convert(obj):
        if isinstance(obj, np.ndarray):
            return recursive_convert(obj.tolist())
        elif isinstance(obj, dict):
            return {k: recursive_convert(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [recursive_convert(item) for item in obj]
        elif isinstance(obj, tuple):
            return tuple(recursive_convert(item) for item in obj)
        elif pd.isnull(obj):
            return None
        else:
            return obj

    try:
        return json.dumps(recursive_convert(value))
    except TypeError as e:
        print(f"TypeError during JSON conversion: {e}")
        return None

def create_db_engine():
    """Create a SQLAlchemy engine for MySQL using PyMySQL."""
    connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}'
    try:
        engine = create_engine(connection_string, pool_pre_ping=True)
        print("Database engine created successfully.")
        return engine
    except Exception as e:
        print(f"Error creating engine: {e}")
        sys.exit(1)

def load_parquet_to_mysql(parquet_file, table_name, engine, chunksize=1000):
    """Load a Parquet file into a MySQL table."""
    try:
        print(f"Loading Parquet file: {parquet_file}")
        df = pd.read_parquet(parquet_file)
        print(f"Parquet file {parquet_file} loaded successfully. Number of rows: {len(df)}")
        
        # Print DataFrame info and sample data for debugging
        print(df.info())
        print(df.head())
        
        if table_name == 'MAPS_SITES':
            # Convert complex data types to JSON strings
            complex_columns = ['category', 'hours', 'MISC', 'relative_results']
            for col in complex_columns:
                if col in df.columns:
                    print(f"Converting column '{col}' to JSON.")
                    df[col] = df[col].apply(convert_to_json)
        elif table_name == 'MAPS_REVIEWS':
            # Identify object columns that might contain dicts or lists
            object_columns = df.select_dtypes(include=['object']).columns
            for col in object_columns:
                # Check if the column contains any dicts, lists, or arrays
                if df[col].apply(lambda x: isinstance(x, (dict, list, np.ndarray))).any():
                    print(f"Converting column '{col}' (contains dicts/lists) to JSON string.")
                    df[col] = df[col].apply(convert_to_json)
        
        # Insert data into MySQL
        df.to_sql(
            name=table_name,
            con=engine,
            if_exists='append',
            index=False,
            chunksize=chunksize,
            method='multi'  # Use multi for batch inserts
        )
        print(f"Successfully loaded {parquet_file} into {table_name}\n")
    except Exception as e:
        print(f"Error loading {parquet_file} into {table_name}: {e}\n")

def main():
    engine = create_db_engine()

    # Load MAPS_SITES first to satisfy foreign key constraints
    load_parquet_to_mysql(MAPS_SITES_PARQUET, 'MAPS_SITES', engine)

    # Then load MAPS_REVIEWS
    load_parquet_to_mysql(MAPS_REVIEWS_PARQUET, 'MAPS_REVIEWS', engine)

if __name__ == "__main__":
    main()


Database engine created successfully.
Loading Parquet file: ../google_maps/metadata-sitios/1.parquet
Parquet file ../google_maps/metadata-sitios/1.parquet loaded successfully. Number of rows: 275001
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              274994 non-null  object 
 1   address           264939 non-null  object 
 2   gmap_id           275001 non-null  object 
 3   description       13155 non-null   object 
 4   latitude          275001 non-null  float64
 5   longitude         275001 non-null  float64
 6   category          272740 non-null  object 
 7   avg_rating        275001 non-null  float64
 8   num_of_reviews    275001 non-null  int64  
 9   price             13450 non-null   object 
 10  hours             192448 non-null  object 
 11  MISC              194972 non-null  object 
 12  state        

---

In [7]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError
from datetime import datetime
import sys

In [9]:

# ------------------------- Configuration -------------------------

# MySQL database configuration
DB_CONFIG = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',       # or your MySQL host
    'port': 3306,              # default MySQL port
    'database': 'google_yelp'
}

# Paths to Parquet files
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'

In [13]:
# ------------------------- Helper Functions -------------------------

def create_tables(cursor):
    """Create tables based on the logical data model."""
    create_sites_table = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        address VARCHAR(500) NOT NULL,
        latitude DECIMAL(9,6) NOT NULL,
        longitude DECIMAL(9,6) NOT NULL,
        avg_rating DECIMAL(2,1),
        num_reviews INT
    );
    """

    create_categories_table = """
    CREATE TABLE IF NOT EXISTS categories (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL UNIQUE
    );
    """

    create_site_categories_table = """
    CREATE TABLE IF NOT EXISTS site_categories (
        site_id VARCHAR(255),
        category_id INT,
        PRIMARY KEY (site_id, category_id),
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE,
        FOREIGN KEY (category_id) REFERENCES categories(category_id) ON DELETE CASCADE
    );
    """

    create_reviews_table = """
    CREATE TABLE IF NOT EXISTS reviews (
        review_id INT AUTO_INCREMENT PRIMARY KEY,
        site_id VARCHAR(255) NOT NULL,
        user_id VARCHAR(50) NOT NULL,
        user_name VARCHAR(255) NOT NULL,
        review_time DATETIME NOT NULL,
        rating TINYINT NOT NULL CHECK (rating BETWEEN 1 AND 5),
        review_text TEXT,
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE
    );
    """

    tables = [create_sites_table, create_categories_table, create_site_categories_table, create_reviews_table]
    for table_query in tables:
        cursor.execute(table_query)
    print("Tables created or verified successfully.")

In [14]:
def epoch_to_datetime(epoch_ms):
    """Convert epoch milliseconds to datetime."""
    return datetime.fromtimestamp(epoch_ms / 1000.0)

In [15]:

# ------------------------- Main Function -------------------------

def main():
    try:
        # Establish MySQL connection
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()
        print("Connected to MySQL database.")

        # # (Optional) Create tables if they don't exist
        # create_tables(cursor)
        # cnx.commit()

        # Create SQLAlchemy engine for bulk operations
        engine_url = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(engine_url, echo=False)

        # ------------------ Process maps_sites ------------------

        print("Processing maps_sites.parquet...")
        sites_df = pd.read_parquet(MAPS_SITES_PARQUET)

        # Drop the 'price' column as per requirement
        if 'price' in sites_df.columns:
            sites_df = sites_df.drop(columns=['price'])

        # Ensure 'category' is a list
        sites_df['category'] = sites_df['category'].apply(lambda x: x if isinstance(x, list) else [])

        # ------------------ Insert Categories ------------------

        # Extract unique categories
        unique_categories = set(cat for sublist in sites_df['category'] for cat in sublist)
        categories_df = pd.DataFrame({'name': list(unique_categories)})

        # Insert categories, ignore duplicates
        try:
            categories_df.to_sql('categories', con=engine, if_exists='append', index=False)
            print(f"Inserted {len(categories_df)} categories.")
        except IntegrityError as e:
            print("Some categories already exist. Continuing...")

        # Fetch category_id mapping
        cursor.execute("SELECT category_id, name FROM categories;")
        category_mapping = {name: cid for cid, name in cursor.fetchall()}

        # ------------------ Insert Sites ------------------

        sites_data = sites_df[['gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews']].copy()
        sites_data.rename(columns={
            'gmap_id': 'site_id',
            'num_of_reviews': 'num_reviews'
        }, inplace=True)

        # Insert sites, ignore duplicates
        try:
            sites_data.to_sql('sites', con=engine, if_exists='append', index=False)
            print(f"Inserted {len(sites_data)} sites.")
        except IntegrityError as e:
            print("Some sites already exist. Continuing...")

        # ------------------ Insert Site_Categories ------------------

        # Prepare site_categories data
        site_categories_records = []
        for _, row in sites_df.iterrows():
            site_id = row['gmap_id']
            categories = row['category']
            for cat in categories:
                cat_id = category_mapping.get(cat)
                if cat_id:
                    site_categories_records.append({'site_id': site_id, 'category_id': cat_id})

        if site_categories_records:
            site_categories_df = pd.DataFrame(site_categories_records).drop_duplicates()
            try:
                site_categories_df.to_sql('site_categories', con=engine, if_exists='append', index=False)
                print(f"Inserted {len(site_categories_df)} site-category relationships.")
            except IntegrityError as e:
                print("Some site-category relationships already exist. Continuing...")
        else:
            print("No site-category relationships to insert.")

        # ------------------ Process maps_reviews ------------------

        print("Processing maps_reviews.parquet...")
        reviews_df = pd.read_parquet(MAPS_REVIEWS_PARQUET)

        # Convert 'time' from epoch ms to datetime
        reviews_df['review_time'] = reviews_df['time'].apply(epoch_to_datetime)

        # Map 'gmap_id' to 'site_id'
        reviews_df.rename(columns={'gmap_id': 'site_id'}, inplace=True)

        # Select relevant columns
        reviews_data = reviews_df[['site_id', 'user_id', 'name', 'review_time', 'rating', 'text']].copy()
        reviews_data.rename(columns={
            'name': 'user_name',
            'text': 'review_text'
        }, inplace=True)

        # Insert reviews
        try:
            reviews_data.to_sql('reviews', con=engine, if_exists='append', index=False)
            print(f"Inserted {len(reviews_data)} reviews.")
        except IntegrityError as e:
            print("Some reviews may already exist or other integrity issues occurred.")

        # Commit all transactions
        cnx.commit()
        print("Data imported successfully.")

    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")
        cnx.rollback()
    except Exception as ex:
        print(f"An error occurred: {ex}")
        cnx.rollback()
    finally:
        if cursor:
            cursor.close()
        if cnx.is_connected():
            cnx.close()
        print("MySQL connection closed.")

# ------------------------- Entry Point -------------------------

if __name__ == "__main__":
    main()

Connected to MySQL database.
Processing maps_sites.parquet...
Inserted 0 categories.
Some sites already exist. Continuing...
No site-category relationships to insert.
Processing maps_reviews.parquet...
Some reviews may already exist or other integrity issues occurred.
Data imported successfully.
MySQL connection closed.


---

In [18]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError, SQLAlchemyError
from datetime import datetime
import sys
import logging

# ------------------------- Configuration -------------------------

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s:%(message)s',
    handlers=[
        logging.FileHandler("data_import.log"),
        logging.StreamHandler(sys.stdout)
    ]
)


# ------------------------- Helper Functions -------------------------

def create_tables(cursor):
    """Create tables based on the logical data model."""
    create_sites_table = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        address VARCHAR(500) NOT NULL,
        latitude DECIMAL(9,6) NOT NULL,
        longitude DECIMAL(9,6) NOT NULL,
        avg_rating DECIMAL(2,1),
        num_reviews INT
    );
    """

    create_categories_table = """
    CREATE TABLE IF NOT EXISTS categories (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL UNIQUE
    );
    """

    create_site_categories_table = """
    CREATE TABLE IF NOT EXISTS site_categories (
        site_id VARCHAR(255),
        category_id INT,
        PRIMARY KEY (site_id, category_id),
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE,
        FOREIGN KEY (category_id) REFERENCES categories(category_id) ON DELETE CASCADE
    );
    """

    create_reviews_table = """
    CREATE TABLE IF NOT EXISTS reviews (
        review_id INT AUTO_INCREMENT PRIMARY KEY,
        site_id VARCHAR(255) NOT NULL,
        user_id VARCHAR(50) NOT NULL,
        user_name VARCHAR(255) NOT NULL,
        review_time DATETIME NOT NULL,
        rating TINYINT NOT NULL CHECK (rating BETWEEN 1 AND 5),
        review_text TEXT,
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE
    );
    """

    tables = [create_sites_table, create_categories_table, create_site_categories_table, create_reviews_table]
    for table_query in tables:
        cursor.execute(table_query)
    logging.info("Tables created or verified successfully.")

def epoch_to_datetime(epoch_ms):
    """Convert epoch milliseconds to datetime."""
    return datetime.fromtimestamp(epoch_ms / 1000.0)

# ------------------------- Main Function -------------------------

def main():
    try:
        # Establish MySQL connection
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()
        logging.info("Connected to MySQL database.")

        # (Optional) Create tables if they don't exist
        create_tables(cursor)
        cnx.commit()

        # Create SQLAlchemy engine for bulk operations
        engine_url = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(engine_url, echo=False)

        # ------------------ Process maps_sites ------------------

        logging.info("Processing maps_sites.parquet...")
        sites_df = pd.read_parquet(MAPS_SITES_PARQUET)

        # Drop the 'price' column as per requirement
        if 'price' in sites_df.columns:
            sites_df = sites_df.drop(columns=['price'])
            logging.info("'price' column dropped due to low data completeness.")

        # Ensure 'category' is a list
        sites_df['category'] = sites_df['category'].apply(lambda x: x if isinstance(x, list) else [])

        # ------------------ Insert Categories ------------------

        # Extract unique categories
        unique_categories = set(cat for sublist in sites_df['category'] for cat in sublist)
        categories_df = pd.DataFrame({'name': list(unique_categories)})

        # Insert categories, ignore duplicates
        try:
            categories_before = pd.read_sql("SELECT name FROM categories;", con=engine)
            new_categories = categories_df[~categories_df['name'].isin(categories_before['name'])]
            if not new_categories.empty:
                new_categories.to_sql('categories', con=engine, if_exists='append', index=False)
                logging.info(f"Inserted {len(new_categories)} new categories.")
            else:
                logging.info("No new categories to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting categories: {e}")

        # Fetch category_id mapping
        cursor.execute("SELECT category_id, name FROM categories;")
        category_mapping = {name: cid for cid, name in cursor.fetchall()}

        # ------------------ Insert Sites ------------------

        sites_data = sites_df[['gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews']].copy()
        sites_data.rename(columns={
            'gmap_id': 'site_id',
            'num_of_reviews': 'num_reviews'
        }, inplace=True)

        # Insert sites, ignore duplicates
        try:
            sites_before = pd.read_sql("SELECT site_id FROM sites;", con=engine)
            new_sites = sites_data[~sites_data['site_id'].isin(sites_before['site_id'])]
            if not new_sites.empty:
                new_sites.to_sql('sites', con=engine, if_exists='append', index=False)
                logging.info(f"Inserted {len(new_sites)} new sites.")
            else:
                logging.info("No new sites to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting sites: {e}")

        # ------------------ Insert Site_Categories ------------------

        # Prepare site_categories data
        site_categories_records = []
        for _, row in sites_df.iterrows():
            site_id = row['gmap_id']
            categories = row['category']
            for cat in categories:
                cat_id = category_mapping.get(cat)
                if cat_id:
                    site_categories_records.append({'site_id': site_id, 'category_id': cat_id})

        if site_categories_records:
            site_categories_df = pd.DataFrame(site_categories_records).drop_duplicates()

            # Determine existing relationships to avoid duplicates
            existing_relationships = pd.read_sql("SELECT site_id, category_id FROM site_categories;", con=engine)
            merged = site_categories_df.merge(existing_relationships, on=['site_id', 'category_id'], how='left', indicator=True)
            new_site_categories = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

            if not new_site_categories.empty:
                new_site_categories.to_sql('site_categories', con=engine, if_exists='append', index=False)
                logging.info(f"Inserted {len(new_site_categories)} new site-category relationships.")
            else:
                logging.info("No new site-category relationships to insert.")
        else:
            logging.info("No site-category relationships to insert.")

        # ------------------ Process maps_reviews ------------------

        logging.info("Processing maps_reviews.parquet...")
        reviews_df = pd.read_parquet(MAPS_REVIEWS_PARQUET)

        # Convert 'time' from epoch ms to datetime
        reviews_df['review_time'] = reviews_df['time'].apply(epoch_to_datetime)

        # Map 'gmap_id' to 'site_id'
        if 'gmap_id' in reviews_df.columns:
            reviews_df.rename(columns={'gmap_id': 'site_id'}, inplace=True)
        else:
            logging.error("'gmap_id' column missing in reviews data.")
            return

        # Select relevant columns
        reviews_data = reviews_df[['site_id', 'user_id', 'name', 'review_time', 'rating', 'text']].copy()
        reviews_data.rename(columns={
            'name': 'user_name',
            'text': 'review_text'
        }, inplace=True)

        # Validate foreign keys (ensure site_id exists in sites)
        valid_site_ids = set(sites_before['site_id'])
        reviews_data = reviews_data[reviews_data['site_id'].isin(valid_site_ids)]

        # Insert reviews
        try:
            reviews_before = pd.read_sql("SELECT user_id, site_id, review_time FROM reviews;", con=engine)
            # Assuming combination of user_id, site_id, and review_time uniquely identifies a review
            reviews_to_insert = reviews_data.merge(
                reviews_before,
                on=['user_id', 'site_id', 'review_time'],
                how='left',
                indicator=True
            )
            new_reviews = reviews_to_insert[reviews_to_insert['_merge'] == 'left_only'].drop(columns=['_merge'])

            if not new_reviews.empty:
                new_reviews.to_sql('reviews', con=engine, if_exists='append', index=False)
                logging.info(f"Inserted {len(new_reviews)} new reviews.")
            else:
                logging.info("No new reviews to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting reviews: {e}")

        # Commit all transactions
        cnx.commit()
        logging.info("Data imported successfully.")

    except mysql.connector.Error as err:
        logging.error(f"MySQL Error: {err}")
        cnx.rollback()
    except Exception as ex:
        logging.error(f"An error occurred: {ex}")
        cnx.rollback()
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx.is_connected():
            cnx.close()
            logging.info("MySQL connection closed.")

# ------------------------- Entry Point -------------------------

if __name__ == "__main__":
    main()

2024-11-06 12:58:53,745 INFO:Connected to MySQL database.
2024-11-06 12:58:53,754 INFO:Tables created or verified successfully.
2024-11-06 12:58:53,758 INFO:Processing maps_sites.parquet...
2024-11-06 12:58:56,930 INFO:'price' column dropped due to low data completeness.
2024-11-06 12:58:57,318 INFO:No new categories to insert.
2024-11-06 12:59:07,724 ERROR:Error inserting sites: (mysql.connector.errors.IntegrityError) 1048 (23000): Column 'address' cannot be null
[SQL: INSERT INTO sites (site_id, name, address, latitude, longitude, avg_rating, num_reviews) VALUES (%(site_id)s, %(name)s, %(address)s, %(latitude)s, %(longitude)s, %(avg_rating)s, %(num_reviews)s)]
[parameters: [{'site_id': '0x88f16e41928ff687:0x883dad4fd048e8f8', 'name': 'Porter Pharmacy', 'address': 'Porter Pharmacy, 129 N Second St, Cochran, GA 31014', 'latitude': 32.3883, 'longitude': -83.3571, 'avg_rating': 4.9, 'num_reviews': 16}, {'site_id': '0x80c2c98c0e3c16fd:0x29ec8a728764fdf9', 'name': 'City Textile', 'address'

In [20]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError, SQLAlchemyError
from datetime import datetime
import sys
import logging

# ------------------------- Configuration -------------------------

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s:%(message)s',
    handlers=[
        logging.FileHandler("data_import.log"),
        logging.StreamHandler(sys.stdout)
    ]
)


# ------------------------- Configuration -------------------------

# MySQL database configuration
DB_CONFIG = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',       # or your MySQL host
    'port': 3306,              # default MySQL port
    'database': 'google_yelp'
}

# Paths to Parquet files
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'

# ------------------------- Helper Functions -------------------------

def create_tables(cursor):
    """Create tables based on the logical data model."""
    create_sites_table = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        address VARCHAR(500) NOT NULL,
        latitude DECIMAL(9,6) NOT NULL,
        longitude DECIMAL(9,6) NOT NULL,
        avg_rating DECIMAL(2,1),
        num_reviews INT
    );
    """

    create_categories_table = """
    CREATE TABLE IF NOT EXISTS categories (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL UNIQUE
    );
    """

    create_site_categories_table = """
    CREATE TABLE IF NOT EXISTS site_categories (
        site_id VARCHAR(255),
        category_id INT,
        PRIMARY KEY (site_id, category_id),
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE,
        FOREIGN KEY (category_id) REFERENCES categories(category_id) ON DELETE CASCADE
    );
    """

    create_reviews_table = """
    CREATE TABLE IF NOT EXISTS reviews (
        review_id INT AUTO_INCREMENT PRIMARY KEY,
        site_id VARCHAR(255) NOT NULL,
        user_id VARCHAR(50) NOT NULL,
        user_name VARCHAR(255) NOT NULL,
        review_time DATETIME NOT NULL,
        rating TINYINT NOT NULL CHECK (rating BETWEEN 1 AND 5),
        review_text TEXT,
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE
    );
    """

    tables = [create_sites_table, create_categories_table, create_site_categories_table, create_reviews_table]
    for table_query in tables:
        cursor.execute(table_query)
    logging.info("Tables created or verified successfully.")

def epoch_to_datetime(epoch_ms):
    """Convert epoch milliseconds to datetime."""
    return datetime.fromtimestamp(epoch_ms / 1000.0)

# ------------------------- Main Function -------------------------

def main():
    try:
        # Establish MySQL connection
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()
        logging.info("Connected to MySQL database.")

        # (Optional) Create tables if they don't exist
        create_tables(cursor)
        cnx.commit()

        # Create SQLAlchemy engine for bulk operations
        engine_url = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(engine_url, echo=False)

        # ------------------ Process maps_sites ------------------

        logging.info("Processing maps_sites.parquet...")
        sites_df = pd.read_parquet(MAPS_SITES_PARQUET)

        # Drop the 'price' column as per requirement
        if 'price' in sites_df.columns:
            sites_df = sites_df.drop(columns=['price'])
            logging.info("'price' column dropped due to low data completeness.")

        # Ensure 'category' is a list
        sites_df['category'] = sites_df['category'].apply(lambda x: x if isinstance(x, list) else [])

        # ------------------ Handle NULL Addresses ------------------

        # Check for NULL addresses
        null_address_count = sites_df['address'].isnull().sum()
        if null_address_count > 0:
            logging.warning(f"Found {null_address_count} records with NULL addresses.")
            # Option 2: Alternatively, fill NULL addresses with a placeholder
            sites_df['address'] = sites_df['address'].fillna('Unknown Address')
            logging.info(f"Filled {null_address_count} NULL addresses with 'Unknown Address'.")
        else:
            logging.info("No records with NULL addresses found.")

        # ------------------ Insert Categories ------------------

        # Extract unique categories
        unique_categories = set(cat for sublist in sites_df['category'] for cat in sublist)
        categories_df = pd.DataFrame({'name': list(unique_categories)})

        # Insert categories, ignore duplicates
        try:
            categories_before = pd.read_sql("SELECT name FROM categories;", con=engine)
            new_categories = categories_df[~categories_df['name'].isin(categories_before['name'])]
            if not new_categories.empty:
                new_categories.to_sql('categories', con=engine, if_exists='append', index=False)
                logging.info(f"Inserted {len(new_categories)} new categories.")
            else:
                logging.info("No new categories to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting categories: {e}")

        # Fetch category_id mapping
        cursor.execute("SELECT category_id, name FROM categories;")
        category_mapping = {name: cid for cid, name in cursor.fetchall()}

        # ------------------ Insert Sites ------------------

        sites_data = sites_df[['gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews']].copy()
        sites_data.rename(columns={
            'gmap_id': 'site_id',
            'num_of_reviews': 'num_reviews'
        }, inplace=True)

        # Insert sites, ignore duplicates
        try:
            sites_before = pd.read_sql("SELECT site_id FROM sites;", con=engine)
            new_sites = sites_data[~sites_data['site_id'].isin(sites_before['site_id'])]
            if not new_sites.empty:
                new_sites.to_sql('sites', con=engine, if_exists='append', index=False)
                logging.info(f"Inserted {len(new_sites)} new sites.")
            else:
                logging.info("No new sites to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting sites: {e}")

        # ------------------ Insert Site_Categories ------------------

        # Prepare site_categories data
        site_categories_records = []
        for _, row in sites_df.iterrows():
            site_id = row['gmap_id']
            categories = row['category']
            for cat in categories:
                cat_id = category_mapping.get(cat)
                if cat_id:
                    site_categories_records.append({'site_id': site_id, 'category_id': cat_id})

        if site_categories_records:
            site_categories_df = pd.DataFrame(site_categories_records).drop_duplicates()

            # Determine existing relationships to avoid duplicates
            existing_relationships = pd.read_sql("SELECT site_id, category_id FROM site_categories;", con=engine)
            new_site_categories = site_categories_df.merge(existing_relationships, on=['site_id', 'category_id'], how='left', indicator=True)
            new_site_categories = new_site_categories[new_site_categories['_merge'] == 'left_only'].drop(columns=['_merge'])

            if not new_site_categories.empty:
                new_site_categories.to_sql('site_categories', con=engine, if_exists='append', index=False)
                logging.info(f"Inserted {len(new_site_categories)} new site-category relationships.")
            else:
                logging.info("No new site-category relationships to insert.")
        else:
            logging.info("No site-category relationships to insert.")

        # ------------------ Process maps_reviews ------------------

        logging.info("Processing maps_reviews.parquet...")
        reviews_df = pd.read_parquet(MAPS_REVIEWS_PARQUET)

        # Convert 'time' from epoch ms to datetime
        reviews_df['review_time'] = reviews_df['time'].apply(epoch_to_datetime)

        # Map 'gmap_id' to 'site_id'
        if 'gmap_id' in reviews_df.columns:
            reviews_df.rename(columns={'gmap_id': 'site_id'}, inplace=True)
        else:
            logging.error("'gmap_id' column missing in reviews data.")
            return

        # Select relevant columns
        reviews_data = reviews_df[['site_id', 'user_id', 'name', 'review_time', 'rating', 'text']].copy()
        reviews_data.rename(columns={
            'name': 'user_name',
            'text': 'review_text'
        }, inplace=True)

        # Convert 'user_id' to string to ensure consistent data types
        reviews_data['user_id'] = reviews_data['user_id'].astype(str)

        # Fetch existing reviews to avoid duplicates
        try:
            reviews_before = pd.read_sql("SELECT user_id, site_id, review_time FROM reviews;", con=engine)
            reviews_before['user_id'] = reviews_before['user_id'].astype(str)

            # Merge to find new reviews
            new_reviews = reviews_data.merge(
                reviews_before,
                on=['user_id', 'site_id', 'review_time'],
                how='left',
                indicator=True
            )
            new_reviews = new_reviews[new_reviews['_merge'] == 'left_only'].drop(columns=['_merge'])

            if not new_reviews.empty:
                new_reviews.to_sql('reviews', con=engine, if_exists='append', index=False)
                logging.info(f"Inserted {len(new_reviews)} new reviews.")
            else:
                logging.info("No new reviews to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting reviews: {e}")

        # Commit all transactions
        cnx.commit()
        logging.info("Data imported successfully.")

    except mysql.connector.Error as err:
        logging.error(f"MySQL Error: {err}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    except Exception as ex:
        logging.error(f"An error occurred: {ex}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx.is_connected():
            cnx.close()
            logging.info("MySQL connection closed.")

# ------------------------- Entry Point -------------------------

if __name__ == "__main__":
    main()


2024-11-06 13:10:09,263 INFO:Connected to MySQL database.
2024-11-06 13:10:09,273 INFO:Tables created or verified successfully.
2024-11-06 13:10:09,275 INFO:Processing maps_sites.parquet...
2024-11-06 13:10:12,485 INFO:'price' column dropped due to low data completeness.
2024-11-06 13:10:12,808 WARNING:Found 10062 records with NULL addresses.
2024-11-06 13:10:12,843 INFO:Filled 10062 NULL addresses with 'Unknown Address'.
2024-11-06 13:10:12,899 INFO:No new categories to insert.
2024-11-06 13:10:24,017 ERROR:Error inserting sites: (mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry '0x88f16e41928ff687:0x883dad4fd048e8f8' for key 'sites.PRIMARY'
[SQL: INSERT INTO sites (site_id, name, address, latitude, longitude, avg_rating, num_reviews) VALUES (%(site_id)s, %(name)s, %(address)s, %(latitude)s, %(longitude)s, %(avg_rating)s, %(num_reviews)s)]
[parameters: [{'site_id': '0x88f16e41928ff687:0x883dad4fd048e8f8', 'name': 'Porter Pharmacy', 'address': 'Porter Pharmacy, 129 

In [22]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError, SQLAlchemyError
from datetime import datetime
import sys
import logging

# ------------------------- Configuration -------------------------

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s:%(message)s',
    handlers=[
        logging.FileHandler("data_import.log"),
        logging.StreamHandler(sys.stdout)
    ]
)

# MySQL database configuration
DB_CONFIG = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',       # or your MySQL host
    'port': 3306,              # default MySQL port
    'database': 'google_yelp'
}

# Paths to Parquet files
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'




# ------------------------- Helper Functions -------------------------

def create_tables(cursor):
    """Create tables based on the logical data model."""
    create_sites_table = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        address VARCHAR(500) NOT NULL,
        latitude DECIMAL(9,6) NOT NULL,
        longitude DECIMAL(9,6) NOT NULL,
        avg_rating DECIMAL(2,1),
        num_reviews INT
    );
    """

    create_categories_table = """
    CREATE TABLE IF NOT EXISTS categories (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL UNIQUE
    );
    """

    create_site_categories_table = """
    CREATE TABLE IF NOT EXISTS site_categories (
        site_id VARCHAR(255),
        category_id INT,
        PRIMARY KEY (site_id, category_id),
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE,
        FOREIGN KEY (category_id) REFERENCES categories(category_id) ON DELETE CASCADE
    );
    """

    create_reviews_table = """
    CREATE TABLE IF NOT EXISTS reviews (
        review_id INT AUTO_INCREMENT PRIMARY KEY,
        site_id VARCHAR(255) NOT NULL,
        user_id VARCHAR(50) NOT NULL,
        user_name VARCHAR(255) NOT NULL,
        review_time DATETIME NOT NULL,
        rating TINYINT NOT NULL CHECK (rating BETWEEN 1 AND 5),
        review_text TEXT,
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE
    );
    """

    tables = [create_sites_table, create_categories_table, create_site_categories_table, create_reviews_table]
    for table_query in tables:
        cursor.execute(table_query)
    logging.info("Tables created or verified successfully.")

def epoch_to_datetime(epoch_ms):
    """Convert epoch milliseconds to datetime."""
    return datetime.fromtimestamp(epoch_ms / 1000.0)

# ------------------------- Main Function -------------------------

def main():
    try:
        # Establish MySQL connection
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()
        logging.info("Connected to MySQL database.")

        # (Optional) Create tables if they don't exist
        create_tables(cursor)
        cnx.commit()

        # Create SQLAlchemy engine for bulk operations
        engine_url = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(engine_url, echo=False)

        # ------------------ Process maps_sites ------------------

        logging.info("Processing maps_sites.parquet...")
        sites_df = pd.read_parquet(MAPS_SITES_PARQUET)

        # Drop the 'price' column as per requirement
        if 'price' in sites_df.columns:
            sites_df = sites_df.drop(columns=['price'])
            logging.info("'price' column dropped due to low data completeness.")

        # Ensure 'category' is a list
        sites_df['category'] = sites_df['category'].apply(lambda x: x if isinstance(x, list) else [])

        # ------------------ Handle NULL Addresses ------------------

        # Check for NULL addresses
        null_address_count = sites_df['address'].isnull().sum()
        if null_address_count > 0:
            logging.warning(f"Found {null_address_count} records with NULL addresses.")
            # Option 1: Remove records with NULL addresses
            # sites_df = sites_df.dropna(subset=['address'])
            # logging.info(f"Removed {null_address_count} records with NULL addresses.")
            # Option 2: Alternatively, fill NULL addresses with a placeholder
            sites_df['address'] = sites_df['address'].fillna('Unknown Address')
            logging.info(f"Filled {null_address_count} NULL addresses with 'Unknown Address'.")
        else:
            logging.info("No records with NULL addresses found.")

        # ------------------ Insert Categories ------------------

        # Extract unique categories
        unique_categories = set(cat for sublist in sites_df['category'] for cat in sublist)
        categories_df = pd.DataFrame({'name': list(unique_categories)})

        # Insert categories, ignore duplicates
        try:
            categories_before = pd.read_sql("SELECT name FROM categories;", con=engine)
            new_categories = categories_df[~categories_df['name'].isin(categories_before['name'])]
            if not new_categories.empty:
                new_categories.to_sql('categories', con=engine, if_exists='append', index=False, chunksize=1000)
                logging.info(f"Inserted {len(new_categories)} new categories.")
            else:
                logging.info("No new categories to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting categories: {e}")

        # Fetch category_id mapping
        cursor.execute("SELECT category_id, name FROM categories;")
        category_mapping = {name: cid for cid, name in cursor.fetchall()}

        # ------------------ Insert Sites ------------------

        sites_data = sites_df[['gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews']].copy()
        sites_data.rename(columns={
            'gmap_id': 'site_id',
            'num_of_reviews': 'num_reviews'
        }, inplace=True)

        # Remove duplicates within new_sites based on site_id
        sites_data = sites_data.drop_duplicates(subset=['site_id'])
        logging.info(f"Total unique sites to consider: {len(sites_data)}")

        # Insert sites, ignore duplicates
        try:
            sites_before = pd.read_sql("SELECT site_id FROM sites;", con=engine)
            new_sites = sites_data[~sites_data['site_id'].isin(sites_before['site_id'])].copy()
            new_sites = new_sites.drop_duplicates(subset=['site_id'])  # Ensure no duplicates within new_sites
            if not new_sites.empty:
                new_sites.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1000)
                logging.info(f"Inserted {len(new_sites)} new sites.")
            else:
                logging.info("No new sites to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting sites: {e}")

        # ------------------ Insert Site_Categories ------------------

        # Prepare site_categories data
        site_categories_records = []
        for _, row in sites_df.iterrows():
            site_id = row['gmap_id']
            categories = row['category']
            for cat in categories:
                cat_id = category_mapping.get(cat)
                if cat_id:
                    site_categories_records.append({'site_id': site_id, 'category_id': cat_id})

        if site_categories_records:
            site_categories_df = pd.DataFrame(site_categories_records).drop_duplicates()
            logging.info(f"Total site-category relationships to consider: {len(site_categories_df)}")

            # Determine existing relationships to avoid duplicates
            existing_relationships = pd.read_sql("SELECT site_id, category_id FROM site_categories;", con=engine)
            new_site_categories = site_categories_df.merge(existing_relationships, on=['site_id', 'category_id'], how='left', indicator=True)
            new_site_categories = new_site_categories[new_site_categories['_merge'] == 'left_only'].drop(columns=['_merge'])

            if not new_site_categories.empty:
                new_site_categories.to_sql('site_categories', con=engine, if_exists='append', index=False, chunksize=1000)
                logging.info(f"Inserted {len(new_site_categories)} new site-category relationships.")
            else:
                logging.info("No new site-category relationships to insert.")
        else:
            logging.info("No site-category relationships to insert.")

        # ------------------ Process maps_reviews ------------------

        logging.info("Processing maps_reviews.parquet...")
        reviews_df = pd.read_parquet(MAPS_REVIEWS_PARQUET)

        # Convert 'time' from epoch ms to datetime
        reviews_df['review_time'] = reviews_df['time'].apply(epoch_to_datetime)

        # Map 'gmap_id' to 'site_id'
        if 'gmap_id' in reviews_df.columns:
            reviews_df.rename(columns={'gmap_id': 'site_id'}, inplace=True)
        else:
            logging.error("'gmap_id' column missing in reviews data.")
            return

        # Select relevant columns
        reviews_data = reviews_df[['site_id', 'user_id', 'name', 'review_time', 'rating', 'text']].copy()
        reviews_data.rename(columns={
            'name': 'user_name',
            'text': 'review_text'
        }, inplace=True)

        # Convert 'user_id' to string to ensure consistent data types
        reviews_data['user_id'] = reviews_data['user_id'].astype(str)

        # Handle potential missing 'user_id's
        missing_user_id_count = reviews_data['user_id'].isnull().sum()
        if missing_user_id_count > 0:
            logging.warning(f"Found {missing_user_id_count} records with NULL 'user_id's.")
            reviews_data['user_id'] = reviews_data['user_id'].fillna('Unknown_User')

        # Fetch existing reviews to avoid duplicates
        try:
            reviews_before = pd.read_sql("SELECT user_id, site_id, review_time FROM reviews;", con=engine)
            reviews_before['user_id'] = reviews_before['user_id'].astype(str)

            # Merge to find new reviews
            new_reviews = reviews_data.merge(
                reviews_before,
                on=['user_id', 'site_id', 'review_time'],
                how='left',
                indicator=True
            )
            new_reviews = new_reviews[new_reviews['_merge'] == 'left_only'].drop(columns=['_merge'])

            # Drop duplicates within new_reviews
            new_reviews = new_reviews.drop_duplicates(subset=['user_id', 'site_id', 'review_time'])

            if not new_reviews.empty:
                new_reviews.to_sql('reviews', con=engine, if_exists='append', index=False, chunksize=1000)
                logging.info(f"Inserted {len(new_reviews)} new reviews.")
            else:
                logging.info("No new reviews to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting reviews: {e}")

        # Commit all transactions
        cnx.commit()
        logging.info("Data imported successfully.")

    except mysql.connector.Error as err:
        logging.error(f"MySQL Error: {err}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    except Exception as ex:
        logging.error(f"An error occurred: {ex}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx.is_connected():
            cnx.close()
            logging.info("MySQL connection closed.")

# ------------------------- Entry Point -------------------------

if __name__ == "__main__":
    main()


2024-11-06 13:24:16,122 INFO:Connected to MySQL database.
2024-11-06 13:24:16,141 INFO:Tables created or verified successfully.
2024-11-06 13:24:16,145 INFO:Processing maps_sites.parquet...
2024-11-06 13:24:21,173 INFO:'price' column dropped due to low data completeness.
2024-11-06 13:24:21,544 WARNING:Found 10062 records with NULL addresses.
2024-11-06 13:24:21,604 INFO:Filled 10062 NULL addresses with 'Unknown Address'.
2024-11-06 13:24:21,689 INFO:No new categories to insert.
2024-11-06 13:24:21,870 INFO:Total unique sites to consider: 248428
2024-11-06 13:26:19,740 INFO:Inserted 210428 new sites.
2024-11-06 13:26:42,898 INFO:No site-category relationships to insert.
2024-11-06 13:26:42,900 INFO:Processing maps_reviews.parquet...
2024-11-06 13:26:56,115 ERROR:Error inserting reviews: (mysql.connector.errors.IntegrityError) 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`google_yelp`.`reviews`, CONSTRAINT `reviews_ibfk_1` FOREIGN KEY (`site_id`) REFER

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\Art\Documents\GitHub\Google-Yelp\venv\lib\site-packages\mysql\connector\connection_cext.py", line 706, in cmd_query
    self._cmysql.query(
_mysql_connector.MySQLInterfaceError: Cannot add or update a child row: a foreign key constraint fails (`google_yelp`.`reviews`, CONSTRAINT `reviews_ibfk_1` FOREIGN KEY (`site_id`) REFERENCES `sites` (`site_id`))

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Art\Documents\GitHub\Google-Yelp\venv\lib\site-packages\sqlalchemy\engine\base.py", line 1936, in _exec_single_context
    self.dialect.do_executemany(
  File "c:\Users\Art\Documents\GitHub\Google-Yelp\venv\lib\site-packages\sqlalchemy\engine\default.py", line 938, in do_executemany
    cursor.executemany(statement, parameters)
  File "c:\Users\Art\Documents\GitHub\Google-Yelp\venv\lib\site-packages\mysql\connector\cursor_cext.py", line 481, in e

2024-11-06 13:26:56,317 INFO:Data imported successfully.
2024-11-06 13:26:56,320 INFO:MySQL connection closed.


In [33]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError, SQLAlchemyError
from datetime import datetime
import sys
import logging

# ------------------------- Configuration -------------------------

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s:%(message)s',
    handlers=[
        logging.FileHandler("data_import.log"),
        logging.StreamHandler(sys.stdout)
    ]
)

# MySQL database configuration
DB_CONFIG = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',       # or your MySQL host
    'port': 3306,              # default MySQL port
    'database': 'google_yelp'
}

# Paths to Parquet files
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'


# ------------------------- Helper Functions -------------------------

def create_tables(cursor):
    """Create tables based on the logical data model."""
    create_sites_table = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255) ,
        address VARCHAR(500) ,
        latitude DECIMAL(9,6) ,
        longitude DECIMAL(9,6) ,
        avg_rating DECIMAL(2,1),
        num_reviews INT
    );
    """

    create_categories_table = """
    CREATE TABLE IF NOT EXISTS categories (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL UNIQUE
    );
    """

    create_site_categories_table = """
    CREATE TABLE IF NOT EXISTS site_categories (
        site_id VARCHAR(255),
        category_id INT,
        PRIMARY KEY (site_id, category_id),
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE,
        FOREIGN KEY (category_id) REFERENCES categories(category_id) ON DELETE CASCADE
    );
    """

    create_reviews_table = """
    CREATE TABLE IF NOT EXISTS reviews (
        review_id INT AUTO_INCREMENT PRIMARY KEY,
        site_id VARCHAR(255) NOT NULL,
        user_id VARCHAR(50) NOT NULL,
        user_name VARCHAR(255) ,
        review_time DATETIME ,
        rating TINYINT CHECK (rating BETWEEN 1 AND 5),
        review_text TEXT,
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE
    );
    """

    tables = [create_sites_table, create_categories_table, create_site_categories_table, create_reviews_table]
    for table_query in tables:
        cursor.execute(table_query)
    logging.info("Tables created or verified successfully.")

def epoch_to_datetime(epoch_ms):
    """Convert epoch milliseconds to datetime."""
    return datetime.fromtimestamp(epoch_ms / 1000.0)

# ------------------------- Main Function -------------------------

def main():
    try:
        # Establish MySQL connection
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()
        logging.info("Connected to MySQL database.")

        # (Optional) Create tables if they don't exist
        create_tables(cursor)
        cnx.commit()

        # Create SQLAlchemy engine for bulk operations
        engine_url = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(engine_url, echo=False)

        # ------------------ Process maps_sites ------------------

        logging.info("Processing maps_sites.parquet...")
        sites_df = pd.read_parquet(MAPS_SITES_PARQUET)

        # Drop the 'price' column as per requirement
        if 'price' in sites_df.columns:
            sites_df = sites_df.drop(columns=['price'])
            logging.info("'price' column dropped due to low data completeness.")

        # Ensure 'category' is a list
        sites_df['category'] = sites_df['category'].apply(lambda x: x if isinstance(x, list) else [])

        # ------------------ Handle NULL Addresses ------------------

        # Check for NULL addresses
        null_address_count = sites_df['address'].isnull().sum()
        if null_address_count > 0:
            logging.warning(f"Found {null_address_count} records with NULL addresses.")
            # Option 1: Remove records with NULL addresses
            # sites_df = sites_df.dropna(subset=['address'])
            # logging.info(f"Removed {null_address_count} records with NULL addresses.")
            # Option 2: Alternatively, fill NULL addresses with a placeholder
            sites_df['address'] = sites_df['address'].fillna('Unknown Address')
            logging.info(f"Filled {null_address_count} NULL addresses with 'Unknown Address'.")
        else:
            logging.info("No records with NULL addresses found.")

        # ------------------ Insert Categories ------------------

        # Extract unique categories
        unique_categories = set(cat for sublist in sites_df['category'] for cat in sublist)
        categories_df = pd.DataFrame({'name': list(unique_categories)})

        # Insert categories, ignore duplicates
        try:
            categories_before = pd.read_sql("SELECT name FROM categories;", con=engine)
            new_categories = categories_df[~categories_df['name'].isin(categories_before['name'])]
            if not new_categories.empty:
                new_categories.to_sql('categories', con=engine, if_exists='append', index=False, chunksize=1000)
                logging.info(f"Inserted {len(new_categories)} new categories.")
            else:
                logging.info("No new categories to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting categories: {e}")

        # Fetch category_id mapping
        cursor.execute("SELECT category_id, name FROM categories;")
        category_mapping = {name: cid for cid, name in cursor.fetchall()}

        # ------------------ Insert Sites ------------------

        sites_data = sites_df[['gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews']].copy()
        sites_data.rename(columns={
            'gmap_id': 'site_id',
            'num_of_reviews': 'num_reviews'
        }, inplace=True)

        # Remove duplicates within new_sites based on site_id
        sites_data = sites_data.drop_duplicates(subset=['site_id'])
        logging.info(f"Total unique sites to consider: {len(sites_data)}")

        # Insert sites, ignore duplicates
        try:
            sites_before = pd.read_sql("SELECT site_id FROM sites;", con=engine)
            new_sites = sites_data[~sites_data['site_id'].isin(sites_before['site_id'])].copy()
            new_sites = new_sites.drop_duplicates(subset=['site_id'])  # Ensure no duplicates within new_sites
            logging.info(f"Total unique new sites to insert: {len(new_sites)}")
            if not new_sites.empty:
                new_sites.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1000)
                logging.info(f"Inserted {len(new_sites)} new sites.")
            else:
                logging.info("No new sites to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting sites: {e}")

        # ------------------ Insert Site_Categories ------------------

        # Prepare site_categories data
        site_categories_records = []
        for _, row in sites_df.iterrows():
            site_id = row['gmap_id']
            categories = row['category']
            for cat in categories:
                cat_id = category_mapping.get(cat)
                if cat_id:
                    site_categories_records.append({'site_id': site_id, 'category_id': cat_id})

        if site_categories_records:
            site_categories_df = pd.DataFrame(site_categories_records).drop_duplicates()
            logging.info(f"Total site-category relationships to consider: {len(site_categories_df)}")

            # Determine existing relationships to avoid duplicates
            existing_relationships = pd.read_sql("SELECT site_id, category_id FROM site_categories;", con=engine)
            new_site_categories = site_categories_df.merge(existing_relationships, on=['site_id', 'category_id'], how='left', indicator=True)
            new_site_categories = new_site_categories[new_site_categories['_merge'] == 'left_only'].drop(columns=['_merge'])

            if not new_site_categories.empty:
                new_site_categories.to_sql('site_categories', con=engine, if_exists='append', index=False, chunksize=1000)
                logging.info(f"Inserted {len(new_site_categories)} new site-category relationships.")
            else:
                logging.info("No new site-category relationships to insert.")
        else:
            logging.info("No site-category relationships to insert.")

        # ------------------ Process maps_reviews ------------------

        logging.info("Processing maps_reviews.parquet...")
        reviews_df = pd.read_parquet(MAPS_REVIEWS_PARQUET)

        # Convert 'time' from epoch ms to datetime
        reviews_df['review_time'] = reviews_df['time'].apply(epoch_to_datetime)

        # Map 'gmap_id' to 'site_id'
        if 'gmap_id' in reviews_df.columns:
            reviews_df.rename(columns={'gmap_id': 'site_id'}, inplace=True)
        else:
            logging.error("'gmap_id' column missing in reviews data.")
            return

        # Select relevant columns
        reviews_data = reviews_df[['site_id', 'user_id', 'name', 'review_time', 'rating', 'text']].copy()
        reviews_data.rename(columns={
            'name': 'user_name',
            'text': 'review_text'
        }, inplace=True)

        # Convert 'user_id' to string to ensure consistent data types
        reviews_data['user_id'] = reviews_data['user_id'].astype(str)

        # Handle potential missing 'user_id's
        missing_user_id_count = reviews_data['user_id'].isnull().sum()
        if missing_user_id_count > 0:
            logging.warning(f"Found {missing_user_id_count} records with NULL 'user_id's.")
            reviews_data['user_id'] = reviews_data['user_id'].fillna('Unknown_User')

        # Fetch existing reviews to avoid duplicates
        try:
            reviews_before = pd.read_sql("SELECT user_id, site_id, review_time FROM reviews;", con=engine)
            reviews_before['user_id'] = reviews_before['user_id'].astype(str)

            # Merge to find new reviews
            new_reviews = reviews_data.merge(
                reviews_before,
                on=['user_id', 'site_id', 'review_time'],
                how='left',
                indicator=True
            )
            new_reviews = new_reviews[new_reviews['_merge'] == 'left_only'].drop(columns=['_merge'])

            # Drop duplicates within new_reviews
            new_reviews = new_reviews.drop_duplicates(subset=['user_id', 'site_id', 'review_time'])

            logging.info(f"Total unique new reviews to insert: {len(new_reviews)}")
            if not new_reviews.empty:
                # Before inserting, check for missing site_ids in sites table
                sites_before_set = set(sites_before['site_id'])
                missing_site_ids = new_reviews[~new_reviews['site_id'].isin(sites_before_set)]['site_id'].unique()
                if len(missing_site_ids) > 0:
                    logging.warning(f"Found {len(missing_site_ids)} unique reviews with missing site_ids in 'sites' table.")

                    # Extract these missing site_ids
                    missing_site_ids_df = pd.DataFrame({'site_id': missing_site_ids})
                    missing_site_ids_df = missing_site_ids_df.drop_duplicates()

                    # Insert placeholder sites for these missing site_ids
                    placeholder_sites = missing_site_ids_df.copy()
                    placeholder_sites['name'] = 'Unknown Site'
                    placeholder_sites['address'] = 'Unknown Address'
                    placeholder_sites['latitude'] = 0.0
                    placeholder_sites['longitude'] = 0.0
                    placeholder_sites['avg_rating'] = None
                    placeholder_sites['num_reviews'] = 0

                    try:
                        placeholder_sites.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1000)
                        logging.info(f"Inserted {len(placeholder_sites)} placeholder sites for missing site_ids.")
                        # Update sites_before_set with newly inserted site_ids
                        sites_before_set.update(missing_site_ids)
                    except SQLAlchemyError as e:
                        logging.error(f"Error inserting placeholder sites: {e}")

                # Refresh sites_before_set to include any newly inserted placeholder sites
                sites_before_set = set(sites_before_set)  # Ensure it's a set
                # Filter out any reviews that still have missing site_ids
                valid_reviews = new_reviews[new_reviews['site_id'].isin(sites_before_set)]

                # Log how many reviews are valid after placeholder insertion
                logging.info(f"Total reviews after ensuring site_ids exist: {len(valid_reviews)}")

                if len(valid_reviews) > 0:
                    valid_reviews.to_sql('reviews', con=engine, if_exists='append', index=False, chunksize=1000)
                    logging.info(f"Inserted {len(valid_reviews)} new reviews.")
                else:
                    logging.info("No valid new reviews to insert after handling missing site_ids.")
            else:
                logging.info("No new reviews to insert.")
        except SQLAlchemyError as e:
            logging.error(f"Error inserting reviews: {e}")

        # Commit all transactions
        cnx.commit()
        logging.info("Data imported successfully.")

    except mysql.connector.Error as err:
        logging.error(f"MySQL Error: {err}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    except Exception as ex:
        logging.error(f"An error occurred: {ex}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx.is_connected():
            cnx.close()
            logging.info("MySQL connection closed.")

# ------------------------- Entry Point -------------------------

if __name__ == "__main__":
    main()

2024-11-06 17:09:28,217 INFO:Connected to MySQL database.
2024-11-06 17:09:28,217 INFO:Connected to MySQL database.
2024-11-06 17:09:28,217 INFO:Connected to MySQL database.
2024-11-06 17:09:28,217 INFO:Connected to MySQL database.
2024-11-06 17:09:28,217 INFO:Connected to MySQL database.
2024-11-06 17:09:28,217 INFO:Connected to MySQL database.
2024-11-06 17:09:28,300 INFO:Tables created or verified successfully.
2024-11-06 17:09:28,300 INFO:Tables created or verified successfully.
2024-11-06 17:09:28,300 INFO:Tables created or verified successfully.
2024-11-06 17:09:28,300 INFO:Tables created or verified successfully.
2024-11-06 17:09:28,300 INFO:Tables created or verified successfully.
2024-11-06 17:09:28,300 INFO:Tables created or verified successfully.
2024-11-06 17:09:28,305 INFO:Processing maps_sites.parquet...
2024-11-06 17:09:28,305 INFO:Processing maps_sites.parquet...
2024-11-06 17:09:28,305 INFO:Processing maps_sites.parquet...
2024-11-06 17:09:28,305 INFO:Processing maps_s

In [32]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError, SQLAlchemyError
from datetime import datetime
import sys
import logging

# ------------------------- Configuration -------------------------

# Configure logging with RotatingFileHandler to prevent log files from growing indefinitely
from logging.handlers import RotatingFileHandler

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create handlers
file_handler = RotatingFileHandler('data_import.log', maxBytes=10**7, backupCount=5)
stream_handler = logging.StreamHandler(sys.stdout)

# Create formatters and add them to handlers
formatter = logging.Formatter('%(asctime)s %(levelname)s:%(message)s')
file_handler.setFormatter(formatter)
stream_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(stream_handler)

# MySQL database configuration
DB_CONFIG = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',       # or your MySQL host
    'port': 3306,              # default MySQL port
    'database': 'google_yelp'
}

# Paths to Parquet files
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'

# ------------------------- Helper Functions -------------------------

def create_tables(cursor):
    """Create tables based on the logical data model."""
    create_sites_table = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255),
        address VARCHAR(500),
        latitude DECIMAL(9,6),
        longitude DECIMAL(9,6),
        avg_rating DECIMAL(2,1),
        num_reviews INT
    );
    """

    create_categories_table = """
    CREATE TABLE IF NOT EXISTS categories (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL UNIQUE
    );
    """

    create_site_categories_table = """
    CREATE TABLE IF NOT EXISTS site_categories (
        site_id VARCHAR(255),
        category_id INT,
        PRIMARY KEY (site_id, category_id),
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE,
        FOREIGN KEY (category_id) REFERENCES categories(category_id) ON DELETE CASCADE
    );
    """

    create_reviews_table = """
    CREATE TABLE IF NOT EXISTS reviews (
        review_id INT AUTO_INCREMENT PRIMARY KEY,
        site_id VARCHAR(255) NOT NULL,
        user_id VARCHAR(50) NOT NULL,
        user_name VARCHAR(255),
        review_time DATETIME,
        rating TINYINT CHECK (rating BETWEEN 1 AND 5),
        review_text TEXT,
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE
    );
    """

    tables = [create_sites_table, create_categories_table, create_site_categories_table, create_reviews_table]
    for table_query in tables:
        cursor.execute(table_query)
    logger.info("Tables created or verified successfully.")

def epoch_to_datetime(epoch_ms):
    """Convert epoch milliseconds to datetime."""
    return datetime.fromtimestamp(epoch_ms / 1000.0)

def normalize_site_id(df, column='site_id'):
    """Normalize the site_id by stripping whitespace and converting to lowercase."""
    df[column] = df[column].astype(str).str.strip().str.lower()
    return df

# ------------------------- Main Function -------------------------

def main():
    try:
        # Establish MySQL connection
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()
        logger.info("Connected to MySQL database.")

        # Create tables if they don't exist
        create_tables(cursor)
        cnx.commit()

        # Create SQLAlchemy engine for bulk operations
        engine_url = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(engine_url, echo=False)

        # ------------------ Process maps_sites ------------------

        logger.info("Processing maps_sites.parquet...")
        sites_df = pd.read_parquet(MAPS_SITES_PARQUET)

        # Drop the 'price' column if it exists
        if 'price' in sites_df.columns:
            sites_df = sites_df.drop(columns=['price'])
            logger.info("'price' column dropped due to low data completeness.")

        # Ensure 'category' is a list
        sites_df['category'] = sites_df['category'].apply(lambda x: x if isinstance(x, list) else [])

        # ------------------ Handle NULL Addresses ------------------

        # Check for NULL addresses
        null_address_count = sites_df['address'].isnull().sum()
        if null_address_count > 0:
            logger.warning(f"Found {null_address_count} records with NULL addresses.")
            # Option 2: Fill NULL addresses with a placeholder
            sites_df['address'] = sites_df['address'].fillna('Unknown Address')
            logger.info(f"Filled {null_address_count} NULL addresses with 'Unknown Address'.")
        else:
            logger.info("No records with NULL addresses found.")

        # ------------------ Insert Categories ------------------

        # Extract unique categories
        unique_categories = set(cat for sublist in sites_df['category'] for cat in sublist)
        categories_df = pd.DataFrame({'name': list(unique_categories)})

        # Insert categories, ignore duplicates
        try:
            categories_before = pd.read_sql("SELECT name FROM categories;", con=engine)
            new_categories = categories_df[~categories_df['name'].isin(categories_before['name'])]
            if not new_categories.empty:
                new_categories.to_sql('categories', con=engine, if_exists='append', index=False, chunksize=1000)
                logger.info(f"Inserted {len(new_categories)} new categories.")
            else:
                logger.info("No new categories to insert.")
        except SQLAlchemyError as e:
            logger.error(f"Error inserting categories: {e}")

        # Fetch category_id mapping
        cursor.execute("SELECT category_id, name FROM categories;")
        category_mapping = {name.lower(): cid for cid, name in cursor.fetchall()}  # Ensure lowercase keys

        # ------------------ Insert Sites ------------------

        sites_data = sites_df[['gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews']].copy()
        sites_data.rename(columns={
            'gmap_id': 'site_id',
            'num_of_reviews': 'num_reviews'
        }, inplace=True)

        # Remove duplicates within new_sites based on site_id
        sites_data = sites_data.drop_duplicates(subset=['site_id'])
        sites_data = normalize_site_id(sites_data, 'site_id')
        logger.info(f"Total unique sites to consider: {len(sites_data)}")

        # Insert sites, ignore duplicates
        try:
            sites_before = pd.read_sql("SELECT site_id FROM sites;", con=engine)
            sites_before_set = set(sites_before['site_id'].str.lower())
            new_sites = sites_data[~sites_data['site_id'].isin(sites_before_set)].copy()
            new_sites = new_sites.drop_duplicates(subset=['site_id'])  # Ensure no duplicates within new_sites
            logger.info(f"Total unique new sites to insert: {len(new_sites)}")
            if not new_sites.empty:
                new_sites.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1000)
                logger.info(f"Inserted {len(new_sites)} new sites.")
            else:
                logger.info("No new sites to insert.")
        except SQLAlchemyError as e:
            logger.error(f"Error inserting sites: {e}")

        # ------------------ Insert Site_Categories ------------------

        # Prepare site_categories data
        site_categories_records = []
        for _, row in sites_df.iterrows():
            site_id = row['gmap_id']
            categories = row['category']
            for cat in categories:
                cat_id = category_mapping.get(cat.lower())  # Use lowercase for matching
                if cat_id:
                    site_categories_records.append({'site_id': site_id.lower(), 'category_id': cat_id})

        if site_categories_records:
            site_categories_df = pd.DataFrame(site_categories_records).drop_duplicates()
            logger.info(f"Total site-category relationships to consider: {len(site_categories_df)}")

            # Determine existing relationships to avoid duplicates
            existing_relationships = pd.read_sql("SELECT site_id, category_id FROM site_categories;", con=engine)
            existing_relationships['site_id'] = existing_relationships['site_id'].str.lower()
            new_site_categories = site_categories_df.merge(existing_relationships, on=['site_id', 'category_id'], how='left', indicator=True)
            new_site_categories = new_site_categories[new_site_categories['_merge'] == 'left_only'].drop(columns=['_merge'])

            if not new_site_categories.empty:
                new_site_categories.to_sql('site_categories', con=engine, if_exists='append', index=False, chunksize=1000)
                logger.info(f"Inserted {len(new_site_categories)} new site-category relationships.")
            else:
                logger.info("No new site-category relationships to insert.")
        else:
            logger.info("No site-category relationships to insert.")

        # ------------------ Process maps_reviews ------------------

        logger.info("Processing maps_reviews.parquet...")
        reviews_df = pd.read_parquet(MAPS_REVIEWS_PARQUET)

        # Convert 'time' from epoch ms to datetime
        reviews_df['review_time'] = reviews_df['time'].apply(epoch_to_datetime)

        # Map 'gmap_id' to 'site_id'
        if 'gmap_id' in reviews_df.columns:
            reviews_df.rename(columns={'gmap_id': 'site_id'}, inplace=True)
        else:
            logger.error("'gmap_id' column missing in reviews data.")
            return

        # Select relevant columns
        reviews_data = reviews_df[['site_id', 'user_id', 'name', 'review_time', 'rating', 'text']].copy()
        reviews_data.rename(columns={
            'name': 'user_name',
            'text': 'review_text'
        }, inplace=True)

        # Normalize 'site_id's in reviews_data
        reviews_data = normalize_site_id(reviews_data, 'site_id')

        # Convert 'user_id' to string to ensure consistent data types
        reviews_data['user_id'] = reviews_data['user_id'].astype(str)

        # Handle potential missing 'user_id's
        missing_user_id_count = reviews_data['user_id'].isnull().sum()
        if missing_user_id_count > 0:
            logger.warning(f"Found {missing_user_id_count} records with NULL 'user_id's.")
            reviews_data['user_id'] = reviews_data['user_id'].fillna('Unknown_User')

        # Fetch existing reviews to avoid duplicates
        try:
            reviews_before = pd.read_sql("SELECT user_id, site_id, review_time FROM reviews;", con=engine)
            reviews_before = normalize_site_id(reviews_before, 'site_id')
            reviews_before['user_id'] = reviews_before['user_id'].astype(str)

            # Merge to find new reviews
            new_reviews = reviews_data.merge(
                reviews_before,
                on=['user_id', 'site_id', 'review_time'],
                how='left',
                indicator=True
            )
            new_reviews = new_reviews[new_reviews['_merge'] == 'left_only'].drop(columns=['_merge'])

            # Drop duplicates within new_reviews
            new_reviews = new_reviews.drop_duplicates(subset=['user_id', 'site_id', 'review_time'])

            logger.info(f"Total unique new reviews to insert: {len(new_reviews)}")
            if not new_reviews.empty:
                # Before inserting, check for missing site_ids in sites table
                sites_before_set = set(sites_before['site_id'].str.lower())
                missing_site_ids = new_reviews[~new_reviews['site_id'].isin(sites_before_set)]['site_id'].unique()
                if len(missing_site_ids) > 0:
                    logger.warning(f"Found {len(missing_site_ids)} unique reviews with missing site_ids in 'sites' table.")

                    # Extract these missing site_ids
                    missing_site_ids_df = pd.DataFrame({'site_id': missing_site_ids})
                    missing_site_ids_df = missing_site_ids_df.drop_duplicates()

                    # Ensure site_ids are still missing before insertion
                    missing_site_ids_df = missing_site_ids_df[~missing_site_ids_df['site_id'].isin(sites_before_set)]
                    logger.info(f"Placeholder sites to insert after verification: {len(missing_site_ids_df)}")

                    # Insert placeholder sites for these missing site_ids
                    placeholder_sites = missing_site_ids_df.copy()
                    placeholder_sites['name'] = 'Unknown Site'
                    placeholder_sites['address'] = 'Unknown Address'
                    placeholder_sites['latitude'] = 0.0
                    placeholder_sites['longitude'] = 0.0
                    placeholder_sites['avg_rating'] = None
                    placeholder_sites['num_reviews'] = 0

                    # Normalize site_ids in placeholder_sites
                    placeholder_sites = normalize_site_id(placeholder_sites, 'site_id')

                    # Re-validate site_ids before insertion to prevent duplicates
                    placeholder_sites = placeholder_sites[~placeholder_sites['site_id'].isin(sites_before_set)]
                    logger.info(f"Valid placeholder sites to insert: {len(placeholder_sites)}")

                    try:
                        if not placeholder_sites.empty:
                            placeholder_sites.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1000)
                            logger.info(f"Inserted {len(placeholder_sites)} placeholder sites for missing site_ids.")
                            # Update sites_before_set with newly inserted site_ids
                            sites_before_set.update(placeholder_sites['site_id'].tolist())
                    except IntegrityError as ie:
                        logger.warning(f"IntegrityError occurred: {ie}. Attempting to skip duplicate site_ids.")
                        # Attempt to insert each placeholder site individually to skip duplicates
                        duplicates = []
                        for _, row in placeholder_sites.iterrows():
                            try:
                                row.to_frame().T.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1)
                            except IntegrityError:
                                duplicates.append(row['site_id'])
                                logger.warning(f"Duplicate site_id skipped: {row['site_id']}")
                        # Remove duplicates from the list
                        valid_placeholder_sites = placeholder_sites[~placeholder_sites['site_id'].isin(duplicates)]
                        # Insert the remaining valid placeholder sites
                        if not valid_placeholder_sites.empty:
                            valid_placeholder_sites.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1000)
                            logger.info(f"Inserted {len(valid_placeholder_sites)} valid placeholder sites.")
                            sites_before_set.update(valid_placeholder_sites['site_id'].tolist())
                    except SQLAlchemyError as e:
                        logger.error(f"Error inserting placeholder sites: {e}")

                # Refresh sites_before_set after inserting placeholders
                sites_before_set = set(sites_before_set)  # Ensure it's a set

                # Filter out any reviews that still have missing site_ids
                valid_reviews = new_reviews[new_reviews['site_id'].isin(sites_before_set)]

                # Log how many reviews are valid after placeholder insertion
                logger.info(f"Total reviews after ensuring site_ids exist: {len(valid_reviews)}")

                if len(valid_reviews) > 0:
                    valid_reviews.to_sql('reviews', con=engine, if_exists='append', index=False, chunksize=1000)
                    logger.info(f"Inserted {len(valid_reviews)} new reviews.")
                else:
                    logger.info("No valid new reviews to insert after handling missing site_ids.")
            else:
                logger.info("No new reviews to insert.")
        except SQLAlchemyError as e:
            logger.error(f"Error inserting reviews: {e}")

        # Commit all transactions
        cnx.commit()
        logger.info("Data imported successfully.")

    except mysql.connector.Error as err:
        logger.error(f"MySQL Error: {err}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    except Exception as ex:
        logger.error(f"An error occurred: {ex}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx.is_connected():
            cnx.close()
            logger.info("MySQL connection closed.")

# ------------------------- Entry Point -------------------------

if __name__ == "__main__":
    main()


2024-11-06 17:08:12,414 INFO:Connected to MySQL database.
2024-11-06 17:08:12,414 INFO:Connected to MySQL database.
2024-11-06 17:08:12,414 INFO:Connected to MySQL database.
2024-11-06 17:08:12,414 INFO:Connected to MySQL database.
2024-11-06 17:08:12,414 INFO:Connected to MySQL database.
2024-11-06 17:08:12,414 INFO:Connected to MySQL database.
2024-11-06 17:08:12,487 INFO:Tables created or verified successfully.
2024-11-06 17:08:12,487 INFO:Tables created or verified successfully.
2024-11-06 17:08:12,487 INFO:Tables created or verified successfully.
2024-11-06 17:08:12,487 INFO:Tables created or verified successfully.
2024-11-06 17:08:12,487 INFO:Tables created or verified successfully.
2024-11-06 17:08:12,487 INFO:Tables created or verified successfully.
2024-11-06 17:08:12,492 INFO:Processing maps_sites.parquet...
2024-11-06 17:08:12,492 INFO:Processing maps_sites.parquet...
2024-11-06 17:08:12,492 INFO:Processing maps_sites.parquet...
2024-11-06 17:08:12,492 INFO:Processing maps_s

KeyboardInterrupt: 

In [30]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError, SQLAlchemyError
from datetime import datetime
import sys
import logging

# ------------------------- Configuration -------------------------

# Configure logging with RotatingFileHandler to prevent log files from growing indefinitely
from logging.handlers import RotatingFileHandler

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create handlers
file_handler = RotatingFileHandler('data_import.log', maxBytes=10**7, backupCount=5)
stream_handler = logging.StreamHandler(sys.stdout)

# Create formatters and add them to handlers
formatter = logging.Formatter('%(asctime)s %(levelname)s:%(message)s')
file_handler.setFormatter(formatter)
stream_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(stream_handler)

# MySQL database configuration
DB_CONFIG = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',       # or your MySQL host
    'port': 3306,              # default MySQL port
    'database': 'google_yelp'
}

# Paths to Parquet files
MAPS_SITES_PARQUET = '../google_maps/metadata-sitios/1.parquet'
MAPS_REVIEWS_PARQUET = '../google_maps/reviews-estados/review-Florida/1.parquet'

# ------------------------- Helper Functions -------------------------

def create_tables(cursor):
    """Create tables based on the logical data model."""
    create_sites_table = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255),
        address VARCHAR(500),
        latitude DECIMAL(9,6),
        longitude DECIMAL(9,6),
        avg_rating DECIMAL(2,1),
        num_reviews INT
    );
    """

    create_categories_table = """
    CREATE TABLE IF NOT EXISTS categories (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL UNIQUE
    );
    """

    create_site_categories_table = """
    CREATE TABLE IF NOT EXISTS site_categories (
        site_id VARCHAR(255),
        category_id INT,
        PRIMARY KEY (site_id, category_id),
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE,
        FOREIGN KEY (category_id) REFERENCES categories(category_id) ON DELETE CASCADE
    );
    """

    create_reviews_table = """
    CREATE TABLE IF NOT EXISTS reviews (
        review_id INT AUTO_INCREMENT PRIMARY KEY,
        site_id VARCHAR(255) NOT NULL,
        user_id VARCHAR(50) NOT NULL,
        user_name VARCHAR(255),
        review_time DATETIME,
        rating TINYINT CHECK (rating BETWEEN 1 AND 5),
        review_text TEXT,
        FOREIGN KEY (site_id) REFERENCES sites(site_id) ON DELETE CASCADE
    );
    """

    tables = [create_sites_table, create_categories_table, create_site_categories_table, create_reviews_table]
    for table_query in tables:
        cursor.execute(table_query)
    logger.info("Tables created or verified successfully.")

def epoch_to_datetime(epoch_ms):
    """Convert epoch milliseconds to datetime."""
    return datetime.fromtimestamp(epoch_ms / 1000.0)

def normalize_site_id(df, column='site_id'):
    """Normalize the site_id by stripping whitespace and converting to lowercase."""
    df[column] = df[column].astype(str).str.strip().str.lower()
    return df

def extract_categories(category_field):
    """
    Extract categories from the category_field.
    Handles different formats: list, comma-separated string, or single string.
    """
    if isinstance(category_field, list):
        return category_field
    elif isinstance(category_field, str):
        # Split by comma and strip whitespace
        return [cat.strip().lower() for cat in category_field.split(',') if cat.strip()]
    else:
        # Unsupported format
        return []

# ------------------------- Main Function -------------------------

def main():
    try:
        # Establish MySQL connection
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()
        logger.info("Connected to MySQL database.")

        # Create tables if they don't exist
        create_tables(cursor)
        cnx.commit()

        # Create SQLAlchemy engine for bulk operations
        engine_url = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(engine_url, echo=False)

        # ------------------ Process maps_sites ------------------

        logger.info("Processing maps_sites.parquet...")
        sites_df = pd.read_parquet(MAPS_SITES_PARQUET)

        # Drop the 'price' column if it exists
        if 'price' in sites_df.columns:
            sites_df = sites_df.drop(columns=['price'])
            logger.info("'price' column dropped due to low data completeness.")

        # Extract and process 'category' field
        if 'category' in sites_df.columns:
            sites_df['category'] = sites_df['category'].apply(extract_categories)
            logger.info("Extracted categories from 'category' field.")
        else:
            logger.warning("'category' column not found in sites data.")
            sites_df['category'] = [[] for _ in range(len(sites_df))]  # Assign empty lists

        # ------------------ Handle NULL Addresses ------------------

        # Check for NULL addresses
        null_address_count = sites_df['address'].isnull().sum()
        if null_address_count > 0:
            logger.warning(f"Found {null_address_count} records with NULL addresses.")
            # Option 2: Fill NULL addresses with a placeholder
            sites_df['address'] = sites_df['address'].fillna('Unknown Address')
            logger.info(f"Filled {null_address_count} NULL addresses with 'Unknown Address'.")
        else:
            logger.info("No records with NULL addresses found.")

        # ------------------ Insert Categories ------------------

        # Extract unique categories
        unique_categories = set(cat for sublist in sites_df['category'] for cat in sublist)
        logger.info(f"Extracted {len(unique_categories)} unique categories from sites data.")
        if unique_categories:
            categories_df = pd.DataFrame({'name': list(unique_categories)})
        else:
            categories_df = pd.DataFrame({'name': []})
            logger.info("No categories found to insert.")

        # Insert categories, ignore duplicates
        try:
            if not categories_df.empty:
                categories_before = pd.read_sql("SELECT name FROM categories;", con=engine)
                # Normalize category names for comparison
                categories_before['name'] = categories_before['name'].str.lower()
                new_categories = categories_df[~categories_df['name'].isin(categories_before['name'])]
                if not new_categories.empty:
                    new_categories.to_sql('categories', con=engine, if_exists='append', index=False, chunksize=1000)
                    logger.info(f"Inserted {len(new_categories)} new categories.")
                else:
                    logger.info("No new categories to insert.")
            else:
                logger.info("No categories to insert.")
        except SQLAlchemyError as e:
            logger.error(f"Error inserting categories: {e}")

        # Fetch category_id mapping
        try:
            cursor.execute("SELECT category_id, name FROM categories;")
            category_mapping = {name.lower(): cid for cid, name in cursor.fetchall()}  # Ensure lowercase keys
            logger.info(f"Fetched {len(category_mapping)} category mappings.")
        except SQLAlchemyError as e:
            logger.error(f"Error fetching category mappings: {e}")
            category_mapping = {}

        # ------------------ Insert Sites ------------------

        sites_data = sites_df[['gmap_id', 'name', 'address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews']].copy()
        sites_data.rename(columns={
            'gmap_id': 'site_id',
            'num_of_reviews': 'num_reviews'
        }, inplace=True)

        # Remove duplicates within new_sites based on site_id
        sites_data = sites_data.drop_duplicates(subset=['site_id'])
        sites_data = normalize_site_id(sites_data, 'site_id')
        logger.info(f"Total unique sites to consider: {len(sites_data)}")

        # Insert sites, ignore duplicates
        try:
            sites_before = pd.read_sql("SELECT site_id FROM sites;", con=engine)
            sites_before_set = set(sites_before['site_id'].str.lower())
            new_sites = sites_data[~sites_data['site_id'].isin(sites_before_set)].copy()
            new_sites = new_sites.drop_duplicates(subset=['site_id'])  # Ensure no duplicates within new_sites
            logger.info(f"Total unique new sites to insert: {len(new_sites)}")
            if not new_sites.empty:
                new_sites.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1000)
                logger.info(f"Inserted {len(new_sites)} new sites.")
            else:
                logger.info("No new sites to insert.")
        except SQLAlchemyError as e:
            logger.error(f"Error inserting sites: {e}")

        # ------------------ Insert Site_Categories ------------------

        # Prepare site_categories data
        site_categories_records = []
        for _, row in sites_df.iterrows():
            site_id = normalize_site_id(pd.DataFrame({'site_id': [row['gmap_id']]}), 'site_id').iloc[0]['site_id']
            categories = row['category']
            for cat in categories:
                cat_id = category_mapping.get(cat.lower())  # Use lowercase for matching
                if cat_id:
                    site_categories_records.append({'site_id': site_id, 'category_id': cat_id})

        if site_categories_records:
            site_categories_df = pd.DataFrame(site_categories_records).drop_duplicates()
            logger.info(f"Total site-category relationships to consider: {len(site_categories_df)}")

            try:
                # Determine existing relationships to avoid duplicates
                existing_relationships = pd.read_sql("SELECT site_id, category_id FROM site_categories;", con=engine)
                existing_relationships['site_id'] = existing_relationships['site_id'].str.lower()
                new_site_categories = site_categories_df.merge(existing_relationships, on=['site_id', 'category_id'], how='left', indicator=True)
                new_site_categories = new_site_categories[new_site_categories['_merge'] == 'left_only'].drop(columns=['_merge'])

                if not new_site_categories.empty:
                    new_site_categories.to_sql('site_categories', con=engine, if_exists='append', index=False, chunksize=1000)
                    logger.info(f"Inserted {len(new_site_categories)} new site-category relationships.")
                else:
                    logger.info("No new site-category relationships to insert.")
            except SQLAlchemyError as e:
                logger.error(f"Error inserting site-category relationships: {e}")
        else:
            logger.info("No site-category relationships to insert.")

        # ------------------ Process maps_reviews ------------------

        logger.info("Processing maps_reviews.parquet...")
        reviews_df = pd.read_parquet(MAPS_REVIEWS_PARQUET)

        # Convert 'time' from epoch ms to datetime
        reviews_df['review_time'] = reviews_df['time'].apply(epoch_to_datetime)

        # Map 'gmap_id' to 'site_id'
        if 'gmap_id' in reviews_df.columns:
            reviews_df.rename(columns={'gmap_id': 'site_id'}, inplace=True)
        else:
            logger.error("'gmap_id' column missing in reviews data.")
            return

        # Select relevant columns
        reviews_data = reviews_df[['site_id', 'user_id', 'name', 'review_time', 'rating', 'text']].copy()
        reviews_data.rename(columns={
            'name': 'user_name',
            'text': 'review_text'
        }, inplace=True)

        # Normalize 'site_id's in reviews_data
        reviews_data = normalize_site_id(reviews_data, 'site_id')

        # Convert 'user_id' to string to ensure consistent data types
        reviews_data['user_id'] = reviews_data['user_id'].astype(str)

        # Handle potential missing 'user_id's
        missing_user_id_count = reviews_data['user_id'].isnull().sum()
        if missing_user_id_count > 0:
            logger.warning(f"Found {missing_user_id_count} records with NULL 'user_id's.")
            reviews_data['user_id'] = reviews_data['user_id'].fillna('Unknown_User')

        # Fetch existing reviews to avoid duplicates
        try:
            reviews_before = pd.read_sql("SELECT user_id, site_id, review_time FROM reviews;", con=engine)
            reviews_before = normalize_site_id(reviews_before, 'site_id')
            reviews_before['user_id'] = reviews_before['user_id'].astype(str)

            # Merge to find new reviews
            new_reviews = reviews_data.merge(
                reviews_before,
                on=['user_id', 'site_id', 'review_time'],
                how='left',
                indicator=True
            )
            new_reviews = new_reviews[new_reviews['_merge'] == 'left_only'].drop(columns=['_merge'])

            # Drop duplicates within new_reviews
            new_reviews = new_reviews.drop_duplicates(subset=['user_id', 'site_id', 'review_time'])

            logger.info(f"Total unique new reviews to insert: {len(new_reviews)}")
            if not new_reviews.empty:
                # Before inserting, check for missing site_ids in sites table
                sites_before_set = set(sites_before['site_id'].str.lower())
                missing_site_ids = new_reviews[~new_reviews['site_id'].isin(sites_before_set)]['site_id'].unique()
                if len(missing_site_ids) > 0:
                    logger.warning(f"Found {len(missing_site_ids)} unique reviews with missing site_ids in 'sites' table.")

                    # Extract these missing site_ids
                    missing_site_ids_df = pd.DataFrame({'site_id': missing_site_ids})
                    missing_site_ids_df = missing_site_ids_df.drop_duplicates()

                    # Ensure site_ids are still missing before insertion
                    missing_site_ids_df = missing_site_ids_df[~missing_site_ids_df['site_id'].isin(sites_before_set)]
                    logger.info(f"Placeholder sites to insert after verification: {len(missing_site_ids_df)}")

                    if not missing_site_ids_df.empty:
                        # Insert placeholder sites for these missing site_ids
                        placeholder_sites = missing_site_ids_df.copy()
                        placeholder_sites['name'] = 'Unknown Site'
                        placeholder_sites['address'] = 'Unknown Address'
                        placeholder_sites['latitude'] = 0.0
                        placeholder_sites['longitude'] = 0.0
                        placeholder_sites['avg_rating'] = None
                        placeholder_sites['num_reviews'] = 0

                        # Normalize site_ids in placeholder_sites
                        placeholder_sites = normalize_site_id(placeholder_sites, 'site_id')

                        # Re-validate site_ids before insertion to prevent duplicates
                        placeholder_sites = placeholder_sites[~placeholder_sites['site_id'].isin(sites_before_set)]
                        logger.info(f"Valid placeholder sites to insert: {len(placeholder_sites)}")

                        try:
                            if not placeholder_sites.empty:
                                placeholder_sites.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1000)
                                logger.info(f"Inserted {len(placeholder_sites)} placeholder sites for missing site_ids.")
                                # Update sites_before_set with newly inserted site_ids
                                sites_before_set.update(placeholder_sites['site_id'].tolist())
                        except IntegrityError as ie:
                            logger.warning(f"IntegrityError occurred: {ie}. Attempting to skip duplicate site_ids.")
                            # Attempt to insert each placeholder site individually to skip duplicates
                            duplicates = []
                            for _, row in placeholder_sites.iterrows():
                                try:
                                    row.to_frame().T.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1)
                                except IntegrityError:
                                    duplicates.append(row['site_id'])
                                    logger.warning(f"Duplicate site_id skipped: {row['site_id']}")
                            # Remove duplicates from the list
                            valid_placeholder_sites = placeholder_sites[~placeholder_sites['site_id'].isin(duplicates)]
                            # Insert the remaining valid placeholder sites
                            if not valid_placeholder_sites.empty:
                                valid_placeholder_sites.to_sql('sites', con=engine, if_exists='append', index=False, chunksize=1000)
                                logger.info(f"Inserted {len(valid_placeholder_sites)} valid placeholder sites.")
                                sites_before_set.update(valid_placeholder_sites['site_id'].tolist())
                        except SQLAlchemyError as e:
                            logger.error(f"Error inserting placeholder sites: {e}")

                # Refresh sites_before_set after inserting placeholders
                sites_before_set = set(sites_before_set)  # Ensure it's a set

                # Filter out any reviews that still have missing site_ids
                valid_reviews = new_reviews[new_reviews['site_id'].isin(sites_before_set)]

                # Log how many reviews are valid after placeholder insertion
                logger.info(f"Total reviews after ensuring site_ids exist: {len(valid_reviews)}")

                if len(valid_reviews) > 0:
                    valid_reviews.to_sql('reviews', con=engine, if_exists='append', index=False, chunksize=1000)
                    logger.info(f"Inserted {len(valid_reviews)} new reviews.")
                else:
                    logger.info("No valid new reviews to insert after handling missing site_ids.")
            else:
                logger.info("No new reviews to insert.")
        except SQLAlchemyError as e:
            logger.error(f"Error inserting reviews: {e}")

        # Commit all transactions
        cnx.commit()
        logger.info("Data imported successfully.")

    except mysql.connector.Error as err:
        logger.error(f"MySQL Error: {err}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    except Exception as ex:
        logger.error(f"An error occurred: {ex}")
        if 'cnx' in locals() and cnx.is_connected():
            cnx.rollback()
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx.is_connected():
            cnx.close()
            logger.info("MySQL connection closed.")

# ------------------------- Entry Point -------------------------

if __name__ == "__main__":
    main()


2024-11-06 16:47:33,955 INFO:Connected to MySQL database.
2024-11-06 16:47:33,955 INFO:Connected to MySQL database.
2024-11-06 16:47:33,955 INFO:Connected to MySQL database.
2024-11-06 16:47:33,955 INFO:Connected to MySQL database.
2024-11-06 16:47:34,036 INFO:Tables created or verified successfully.
2024-11-06 16:47:34,036 INFO:Tables created or verified successfully.
2024-11-06 16:47:34,036 INFO:Tables created or verified successfully.
2024-11-06 16:47:34,036 INFO:Tables created or verified successfully.
2024-11-06 16:47:34,041 INFO:Processing maps_sites.parquet...
2024-11-06 16:47:34,041 INFO:Processing maps_sites.parquet...
2024-11-06 16:47:34,041 INFO:Processing maps_sites.parquet...
2024-11-06 16:47:34,041 INFO:Processing maps_sites.parquet...
2024-11-06 16:47:36,329 INFO:'price' column dropped due to low data completeness.
2024-11-06 16:47:36,329 INFO:'price' column dropped due to low data completeness.
2024-11-06 16:47:36,329 INFO:'price' column dropped due to low data complete